In [86]:
library(R.matlab)
library(wavelets)
library(tuneR)
library(seewave)
library(data.table)
library(constellation)
library(GENEAread)

Loading required package: bitops

Loading required package: mmap



In [202]:
baseName <- "frame"
wavesName = c("8PSK", "16QAM", "64QAM", "BPSK", "CPFSK", "GFSK", "QPSK", "PAM4")
dffile <- data.frame()

for (w in wavesName) {
    for(i in 1:1) {
        n <- formatC(i, width=3, flag="0")
        name <- paste(baseName, w, n, ".mat", sep="")
        pathfile <- paste("../dataset/origin/", name, sep="")
        signal <- readMat(pathfile)
        values <- signal['frame']$frame
        
        #Time Domain
        t=seq(0,1023)
        s = Mod(values) * cos((pi*2.5*t) + Arg(values))
        
        #Frequency Domain (Fourier)
        f_s <- fft(s)
        
        #Frequency Domain (Wavelets)
        s_wavelets <- dwt(s, filter="la8", n.levels=3, boundary="periodic", fast=TRUE)
        s_ww <- c(s_wavelets@W$W1, s_wavelets@W$W2, s_wavelets@W$W3, c(1:128))
        
        #Zero-Cossing
        #zc <- approxfun(s, t)
        #s_wave <- Wave(left=s, samp.rate=2500000000, bit=8)
        #s_zc <- zc(s_wave, plot=FALSE)
        #str(s_zc)
        #print(s_zc)
        
        #autocorrelation
        ac <- acf (s, lag = 1023, plot=FALSE)
        
        #Constellation Shape Features
        #tt <- seq(from = as.POSIXct("00:00:00", format="%H:%M:%S", tz="UTC"), length.out = 1024, by = "mins")
        #DF = data.table( ID = 1, TIME = tt, VALUE = s)
        #DT_C_SHAPE <- value_change(DF, value = 1, direction = "down", window_hours = 18, join_key = "ID", time_var = "TIME", value_var = "VALUE.V1", mult = "all")
        #constellate(tt, DF, window_hours = 6, join_key = "ID", time_var = "TIME", event_name = "VALUE.V1", mult = "first")
        
        
        #S Transform
        s_s <- stft(s, plot = FALSE, reassign = TRUE, win = 100)
        print(s_s$values)
        
        dfrow <- data.frame(file = name, wave = w, bandpass=values, timeDomain=s,
                            fourierDomain=f_s, waveletsDomain=s_ww, autocorrelation=ac$acf)
        dffile <- rbind(dffile, dfrow)
    }
}

write.csv(dffile, "../dataset/transform/data.csv", row.names = TRUE)

           [,1]      [,2]      [,3]      [,4]      [,5]      [,6]       [,7]
 [1,] 0.1128012 0.7080479 1.2886552 1.1903829 0.7910162 1.3378045 0.08489817
 [2,] 0.4629107 0.8184919 0.2248477 0.4208706 0.3872901 1.3058728 1.19841508
 [3,] 0.8664510 1.5108247 0.7700520 1.0691108 0.9168746 0.6481263 0.38595573
 [4,] 0.3329138 0.6282788 1.2997008 0.3435753 1.0903595 1.4765292 1.40100889
 [5,] 0.4088700 0.5555627 1.7793118 2.3309472 2.2471452 1.1928680 0.96757842
 [6,] 0.6253644 1.0303417 0.7374185 1.4443735 3.1737659 1.5631773 0.32940501
 [7,] 0.9342921 1.7148217 0.8466819 1.3122319 1.0282776 1.0371131 1.22752190
 [8,] 0.1678416 1.5417482 1.3630503 1.9960701 1.8052650 0.9554376 0.93911593
 [9,] 0.6127106 1.3602277 0.7477008 0.7159407 0.1388285 0.3697104 0.57021338
[10,] 0.5552231 1.5466436 1.0827535 0.6852742 0.9744842 0.8596052 1.96417646
[11,] 0.2995132 0.9597826 0.8959582 0.2464258 0.3243037 1.1714963 0.38359279
[12,] 0.1749811 1.0050143 0.9646839 0.7485741 0.8482366 0.6003375 1.77097120